# 📘 Agentic 架构 7：黑板系统 (Blackboard Systems)

欢迎来到我们关于 Agentic 架构系列的第七个笔记本。今天，我们将探讨**黑板系统**（Blackboard System），这是一种强大且高度灵活的模式，用于协调多个专长代理。这种架构模仿了一群人类专家围绕物理黑板协作解决复杂问题的场景。

与刚性的、预定义的代理移交序列不同，黑板系统拥有一个中央共享数据存储（即“黑板”），代理可以在其中读取问题的当前状态，并写入自己的贡献。一个动态的**Controller**（控制器）观察黑板，并根据推进解决方案所需的内容决定下一步激活哪个专长代理。这允许出现机会主义的、涌现式的工作流程。

为了突出其独特优势，我们将把它与之前构建的**顺序式多代理系统**进行比较。我们将向两个系统呈现一个复杂的金融查询，其中最优路径并非简单的 A → B → C 序列。我们将展示刚性的顺序代理如何遵循次优路径，而黑板系统的动态控制器则以更逻辑、更数据驱动的顺序激活代理，从而产生更高效且更连贯的分析。

### 定义
**黑板系统**（Blackboard System）是一种多代理架构，其中多个专长代理通过读取和写入一个名为“黑板”的共享中央数据存储库来进行协作。一个控制器或调度器根据黑板上解决方案的演变状态，动态决定下一个应该由哪个代理执行。

### 高层工作流程

1.  **共享内存（黑板）：** 一个中央数据结构保存问题的当前状态，包括用户请求、中间发现和部分解决方案。
2.  **专长代理：** 一组独立的代理，每个代理拥有特定专长，不断监控黑板。
3.  **控制器：** 一个中央“控制器”代理同样监控黑板。它的任务是分析当前状态，并决定哪个专长代理最适合做出下一步贡献。
4.  **机会主义激活：** 控制器激活选定的代理。该代理从黑板读取相关数据，执行其任务，并将发现写回黑板。
5.  **迭代：** 该过程重复进行，控制器以动态序列激活不同代理，直到它判断黑板上的解决方案已完整。

### 何时使用 / 应用场景
*   **复杂、非结构化问题：** 适用于解决方案路径事先未知、需要涌现式、机会主义策略的问题（例如，复杂诊断、科学研究发现）。
*   **多模态系统：** 协调处理不同数据类型（文本、图像、代码）的代理的绝佳方式，因为它们都可以将发现发布到共享黑板上。
*   **动态意义构建：** 需要从众多不同、异步来源合成信息的情景。

### 优势与劣势
*   **优势：**
    *   **灵活性与适应性：** 工作流程并非硬编码，而是根据问题涌现而成，使系统高度适应性强。
    *   **模块化：** 非常容易添加或移除专长代理，而无需重新设计整个系统架构。
*   **劣势：**
    *   **控制器复杂性：** 整个系统的智能高度依赖控制器的复杂程度。一个简单的控制器可能导致低效或循环行为。
    *   **调试挑战：** 工作流程的非线性、涌现特性有时会比简单的顺序流程更难追踪和调试。

## 第零阶段：基础与设置

我们将从标准设置流程开始：安装库并为 Nebius、LangSmith 和 Tavily 配置 API 密钥。

### 步骤 0.1：安装核心库

**我们将要做的事情：**
我们将为本项目系列安装标准的核心库套件。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### 步骤 0.2：导入库并设置密钥

**我们将要做的事情：**
我们将导入必要的模块，并从 `.env` 文件中加载我们的 API 密钥。

**所需操作：** 请在本目录下创建一个 `.env` 文件，并填写您的密钥：
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [2]:
import os
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain components
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Blackboard (Nebius)"

for key in ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：基线 - 顺序式多代理系统（修正版）

为了理解黑板系统的灵活性，我们首先需要一个正确运行的顺序式系统。原始版本失败的原因是专长代理没有使用前一步骤的输出。我们将通过确保每个代理从状态中接收必要的上下文来修正这一问题。

### 步骤 1.1：构建（修正版）顺序式团队

**我们将要做的事情：**
我们将定义专长代理，使其明确使用前任代理的输出，然后将它们以固定的线性序列连接起来。

In [3]:
console = Console()
# Using a more capable model to handle complex instructions better
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=2)

# State for the sequential agent
class SequentialState(TypedDict):
    user_request: str
    news_report: Optional[str]
    technical_report: Optional[str]
    financial_report: Optional[str]
    final_report: Optional[str]

# --- CORRECTED SPECIALIST NODES FOR SEQUENTIAL AGENT ---
# The key change is that each agent now gets context from previous steps, not just the original request.

def news_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING NEWS ANALYST ---")
    prompt = f"Your task is to act as an expert News Analyst. Find the latest major news about the topic in the user's request and provide a concise summary.\n\nUser Request: {state['user_request']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"news_report": result.content}

def technical_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING TECHNICAL ANALYST ---")
    # This agent now uses the news report as context.
    prompt = f"Your task is to act as an expert Technical Analyst. Based on the following news report, conduct a technical analysis of the company's stock.\n\nNews Report:\n{state['news_report']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"technical_report": result.content}

def financial_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING FINANCIAL ANALYST ---")
    # This agent also uses the news report as context.
    prompt = f"Your task is to act as an expert Financial Analyst. Based on the following news report, analyze the company's recent financial performance.\n\nNews Report:\n{state['news_report']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"financial_report": result.content}


def report_writer_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING REPORT WRITER ---")
    prompt = f"""You are an expert report writer. Your task is to synthesize the information from the News, Technical, and Financial analysts into a single, cohesive report that directly answers the user's original request.

User Request: {state['user_request']}

Here are the reports to combine:
---
News Report: {state['news_report']}
---
Technical Report: {state['technical_report']}
---
Financial Report: {state['financial_report']}
"""
    report = llm.invoke(prompt).content
    return {"final_report": report}

# Build the sequential graph
seq_graph_builder = StateGraph(SequentialState)
seq_graph_builder.add_node("news", news_analyst_node_seq)
seq_graph_builder.add_node("tech", technical_analyst_node_seq)
seq_graph_builder.add_node("finance", financial_analyst_node_seq)
seq_graph_builder.add_node("writer", report_writer_node_seq)

# The rigid, hardcoded sequence
seq_graph_builder.set_entry_point("news")
seq_graph_builder.add_edge("news", "tech")
seq_graph_builder.add_edge("tech", "finance")
seq_graph_builder.add_edge("finance", "writer")
seq_graph_builder.add_edge("writer", END)

sequential_app = seq_graph_builder.compile()
print("Corrected sequential multi-agent system compiled successfully.")

Corrected sequential multi-agent system compiled successfully.


### 步骤 1.2：在动态问题上测试顺序式代理

现在顺序式代理已经能够正确传递上下文，让我们观察它的行为。它将产生一份更连贯的报告，但其*过程*仍然低效，并且无法遵循条件逻辑。

In [4]:
dynamic_query = "Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative)."

console.print(f"[bold yellow]Testing CORRECTED SEQUENTIAL agent on a dynamic query:[/bold yellow]\n'{dynamic_query}'\n")

# Run the graph
final_seq_output = sequential_app.invoke({"user_request": dynamic_query})

console.print("\n--- [bold red]Final Report from Sequential Agent[/bold red] ---")
console.print(Markdown(final_seq_output['final_report']))

Testing CORRECTED SEQUENTIAL agent on a dynamic query:
'Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).'


--- (Sequential) CALLING NEWS ANALYST ---
--- (Sequential) CALLING TECHNICAL ANALYST ---
--- (Sequential) CALLING FINANCIAL ANALYST ---
--- (Sequential) CALLING REPORT WRITER ---



--- Final Report from Sequential Agent ---


### Comprehensive Analysis of Nvidia Following Latest News

**User Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**1. News Summary:**
Recent major news concerning Nvidia revolves around the announcement of their next-generation AI chip architecture, codenamed "Rubin." This news has generated significant positive sentiment in the market, with analysts highlighting the potential for continued dominance in the AI hardware space. The announcement points to a roadmap extending beyond the current "Blackwell" chips, suggesting sustained innovation and growth. This news is overwhelmingly positive.

**2. Technical Analysis:**
Following the positive news, Nvidia's stock (NVDA) has shown strong bullish indicators. The stock price has broken through previous resistance levels and is trading well above its 50-day and 200-day moving averages. Trading volume has been high on up-days, confirming the bullish trend. Key indicators like the Relative Strength Index (RSI) are in overbought territory, suggesting a possible short-term pullback, but the overall momentum remains strongly positive.

**3. Financial Analysis:**
Nvidia's recent financial performance has been exceptionally strong, driven by surging demand for its data center GPUs. The company's latest quarterly earnings report showed revenue and net income figures that significantly beat analyst expectations. Gross margins have expanded, and the company has provided a very optimistic forecast for the upcoming quarter, further bolstering investor confidence.

**Conclusion:**
The combination of groundbreaking product announcements, positive market sentiment, strong technical stock performance, and record-breaking financial results paints a very bullish picture for Nvidia. While the user's request specified an 'either/or' analysis based on sentiment, this report includes all analyses for completeness, confirming the company's robust position from multiple perspectives.

**（修正版）输出讨论：**
代理现在能够产生一份完整且逻辑连贯的报告。然而，执行轨迹 `News → Technical → Financial` 揭示了它的根本缺陷。它执行了**技术分析和财务分析两者**，完全忽略了用户的条件请求（“either... or...”）。这种做法效率低下，并展示了我们希望通过黑板架构来解决的刚性问题。

## 第二阶段：高级方法 - 黑板系统（修正版）

现在，我们将构建黑板系统。修复原始版本循环行为的关键在于，为**控制器**（Controller）设计一个更加智能的提示，使其意识到自己作为有状态规划器的角色。

### 步骤 2.1：定义黑板和（修正版）控制器

**我们将要做的事情：**
1.  **黑板状态：** 定义一个 `BlackboardState` 用于共享内存。
2.  **专长代理：** 定义专长节点。这些节点将与我们之前的代理类似。
3.  **控制器（修正版）：** 创建一个健壮的 `controller_node`（控制器节点），其提示明确要求对已完成的步骤和剩余目标进行推理。这是本次最关键的改动。

In [5]:
# The Blackboard State holds all information
class BlackboardState(TypedDict):
    user_request: str
    # The central blackboard where agents post their findings as strings
    blackboard: List[str]
    # List of available agents for the controller to choose from
    available_agents: List[str]
    # The controller's next decision
    next_agent: Optional[str]

# Pydantic model for the Controller's decision
# CORRECTION: Added the list of available agents to the field description to guide the LLM's choice.
class ControllerDecision(BaseModel):
    next_agent: str = Field(description="The name of the next agent to call. Must be one of ['News Analyst', 'Technical Analyst', 'Financial Analyst', 'Report Writer'] or 'FINISH'.")
    reasoning: str = Field(description="A brief reason for choosing the next agent.")

# Reusable factory for creating specialist agents for the blackboard
def create_blackboard_specialist(persona: str, agent_name: str):
    system_prompt = f"""You are an expert specialist agent: a {persona}.
Your task is to contribute to a larger goal by performing your specific function.
Read the initial User Request and the current Blackboard for context.
Use your tools to find the required information.
Finally, post your concise markdown report back to the blackboard. Your report should be signed with your name '{agent_name}'.
"""
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "User Request: {user_request}\n\nBlackboard (previous reports):\n{blackboard_str}")
    ])
    agent = prompt_template | llm.bind_tools([search_tool])

    def specialist_node(state: BlackboardState):
        console.print(f"--- (Blackboard) AGENT '{agent_name}' is working... ---")
        blackboard_str = "\n---\n".join(state["blackboard"])
        result = agent.invoke({"user_request": state["user_request"], "blackboard_str": blackboard_str})
        report = f"**Report from {agent_name}:**\n{result.content}"
        # Append the new report to the list of blackboard entries
        return {"blackboard": state["blackboard"] + [report]}
    return specialist_node

# Create the specialist agent nodes
news_analyst_bb = create_blackboard_specialist("News Analyst", "News Analyst")
technical_analyst_bb = create_blackboard_specialist("Technical Analyst", "Technical Analyst")
financial_analyst_bb = create_blackboard_specialist("Financial Analyst", "Financial Analyst")
report_writer_bb = create_blackboard_specialist("Report Writer who synthesizes a final answer from the blackboard", "Report Writer")

# --- THE CORRECTED, INTELLIGENT CONTROLLER NODE ---
# This is the most important fix. The prompt is now much more sophisticated.
def controller_node(state: BlackboardState):
    console.print("--- CONTROLLER: Analyzing blackboard... ---")

    # Use a structured output LLM to make the decision
    controller_llm = llm.with_structured_output(ControllerDecision)

    blackboard_content = "\n\n".join(state['blackboard'])
    agent_list = state['available_agents']

    # The new prompt is state-aware and goal-oriented.
    prompt = f"""You are the central controller of a multi-agent system. Your job is to analyze the shared blackboard and the original user request to decide which specialist agent should run next.

**Original User Request:**
{state['user_request']}

**Current Blackboard Content:**
---
{blackboard_content if blackboard_content else "The blackboard is currently empty."}
---

**Available Specialist Agents:**
{', '.join(agent_list)}

**Your Task:**
1.  Read the user request and the current blackboard content carefully.
2.  Determine what the *next logical step* is to move closer to a complete answer.
3.  Choose the single best agent to perform that step from the list of available agents.
4.  If the user's request has been fully addressed and a final report has been written, choose 'FINISH'. Do not finish until a "Report Writer" has provided a final, synthesized answer.

Provide your decision in the required format.
"""
    decision_result = controller_llm.invoke(prompt)
    console.print(f"--- CONTROLLER: Decision is to call '{decision_result.next_agent}'. Reason: {decision_result.reasoning} ---")

    # The dictionary returned here updates the 'next_agent' key in the graph's state
    return {"next_agent": decision_result.next_agent}

print("Blackboard components and corrected Controller node defined.")

Blackboard components and corrected Controller node defined.


### 步骤 2.2：构建黑板图

现在，我们将这些组件连接成一个动态图。控制器充当中央路由器。在任何专长代理运行之后，控制权始终返回到控制器，由它决定下一步行动。

In [6]:
bb_graph_builder = StateGraph(BlackboardState)

# Add all nodes to the graph
bb_graph_builder.add_node("Controller", controller_node)
bb_graph_builder.add_node("News Analyst", news_analyst_bb)
bb_graph_builder.add_node("Technical Analyst", technical_analyst_bb)
bb_graph_builder.add_node("Financial Analyst", financial_analyst_bb)
bb_graph_builder.add_node("Report Writer", report_writer_bb)

bb_graph_builder.set_entry_point("Controller")

# This function defines the dynamic routing logic based on the Controller's decision
def route_to_agent(state: BlackboardState):
    return state["next_agent"]

# The conditional edges route from the Controller to the chosen specialist or to the end
bb_graph_builder.add_conditional_edges(
    "Controller",
    route_to_agent,
    {
        "News Analyst": "News Analyst",
        "Technical Analyst": "Technical Analyst",
        "Financial Analyst": "Financial Analyst",
        "Report Writer": "Report Writer",
        "FINISH": END
    }
)

# After any specialist runs, control always returns to the Controller for the next decision
bb_graph_builder.add_edge("News Analyst", "Controller")
bb_graph_builder.add_edge("Technical Analyst", "Controller")
bb_graph_builder.add_edge("Financial Analyst", "Controller")
bb_graph_builder.add_edge("Report Writer", "Controller")

blackboard_app = bb_graph_builder.compile()
print("Blackboard system compiled successfully.")

Blackboard system compiled successfully.


## 第三阶段：面对面对比

让我们在新构建的黑板系统上运行相同的动态任务，并观察其智能的工作流程。

In [7]:
console.print(f"[bold green]Testing BLACKBOARD system on the same dynamic query:[/bold green]\n'{dynamic_query}'\n")

agent_list = ["News Analyst", "Technical Analyst", "Financial Analyst", "Report Writer"]
initial_bb_input = {"user_request": dynamic_query, "blackboard": [], "available_agents": agent_list}

# We use stream to observe the step-by-step process
final_bb_output = None
for chunk in blackboard_app.stream(initial_bb_input, {"recursion_limit": 10}):
    final_bb_output = chunk
    console.print("\n--- [bold purple]Current Blackboard State[/bold purple] ---")
    # Pretty print each report on the blackboard
    for i, report in enumerate(final_bb_output.get('blackboard', [])):
        console.print(f"--- Report {i+1} ---")
        console.print(Markdown(report))
    console.print("\n")

console.print("\n--- [bold green]Final Report from Blackboard System[/bold green] ---")
# The final report is the last item posted to the blackboard by the writer
final_report_content = final_bb_output['blackboard'][-1]
console.print(Markdown(final_report_content))

Testing BLACKBOARD system on the same dynamic query:
'Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).'


--- CONTROLLER: Analyzing blackboard... ---
--- CONTROLLER: Decision is to call 'News Analyst'. Reason: The blackboard is empty, so the first step is to gather the latest news about Nvidia as requested by the user. ---



--- Current Blackboard State ---



--- (Blackboard) AGENT 'News Analyst' is working... ---
--- CONTROLLER: Analyzing blackboard... ---
--- CONTROLLER: Decision is to call 'Technical Analyst'. Reason: The news report on the blackboard is positive, mentioning a new chip architecture and market dominance. According to the user's request, a technical analysis is the correct next step for positive news. ---



--- Current Blackboard State ---


...

**Report from Technical Analyst:**
Based on the positive news regarding Nvidia's accelerated roadmap, a technical analysis of NVDA stock shows strong bullish momentum. The stock is currently in a clear uptrend, trading significantly above its 50-day and 200-day simple moving averages (SMAs). The announcement created a bullish gap on the price chart, which has held, indicating strong buying pressure. The Relative Strength Index (RSI) is high, suggesting the stock is in overbought territory, but this is common for a stock in a strong uptrend. Key support is now established at the top of the recent price gap.

--- Report 3 ---


**Report from Report Writer:**
### Final Analysis of Nvidia Based on Recent News

**Initial Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**Process Summary:**
1.  **News Analysis:** The latest major news for Nvidia is the announcement of its next-generation "Rubin" AI chip platform, which has generated highly positive market sentiment.
2.  **Conditional Action:** As per the user's request, the positive news sentiment triggered a technical analysis.
3.  **Technical Analysis:** The technical outlook for NVDA stock is strongly bullish. The stock is in a clear uptrend, trading above key moving averages, with significant buying pressure confirmed by a recent price gap.

**Conclusion:** Following the user's instructions, an analysis of recent positive news about Nvidia's accelerated innovation led to a technical review, which confirms a strong bullish momentum for the company's stock.


--- Final Report from Blackboard System ---


**Report from Report Writer:**
### Final Analysis of Nvidia Based on Recent News

**Initial Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**Process Summary:**
1.  **News Analysis:** The latest major news for Nvidia is the announcement of its next-generation "Rubin" AI chip platform, which has generated highly positive market sentiment.
2.  **Conditional Action:** As per the user's request, the positive news sentiment triggered a technical analysis.
3.  **Technical Analysis:** The technical outlook for NVDA stock is strongly bullish. The stock is in a clear uptrend, trading above key moving averages, with significant buying pressure confirmed by a recent price gap.

**Conclusion:** Following the user's instructions, an analysis of recent positive news about Nvidia's accelerated innovation led to a technical review, which confirms a strong bullish momentum for the company's stock.

**（修正版）输出讨论：**
成功！`GraphRecursionError` 已经消失。执行轨迹展示了一个远更智能的过程：

1.  **控制器启动：** 控制器启动，看到黑板为空，正确决定首先调用 **News Analyst**（新闻分析师）。
2.  **新闻分析师运行：** 新闻分析师查找最新新闻，并将其报告发布到黑板上。
3.  **控制器重新评估：** 控制权返回到控制器。它读取新闻分析师的报告，理解其中的情绪倾向，并遵循用户的逻辑。它智能地决定调用合适的下一个分析师（**技术分析师**或**财务分析师**），完全跳过另一个。
4.  **专长代理运行：** 被选中的分析师执行其任务，并将其报告添加到黑板上。
5.  **控制器完成：** 控制器看到所有必要的分析已完成，便调用 **Report Writer**（报告撰写器）来合成最终答案。
6.  **最终调用：** 在撰写器发布最终报告后，控制器看到这一点并决定 **FINISH**（结束）。

这种动态的、机会主义的工作流程是正常运行的黑板系统的标志性特征。它完美遵循了用户复杂的条件逻辑，从而节省了时间和资源。

## 第四阶段：定量评估

为了正式化比较，我们将使用 LLM-as-a-Judge（以 LLM 作为评判者）来对两个系统在指令遵循度和过程效率方面的表现进行评分。

In [8]:
class ProcessLogicEvaluation(BaseModel):
    """Schema for evaluating an agent's logical process."""
    instruction_following_score: int = Field(description="Score 1-10 on how well the agent followed the user's specific conditional instructions (e.g., the 'either/or' logic).")
    process_efficiency_score: int = Field(description="Score 1-10 on whether the agent took the most direct path and avoided unnecessary work.")
    justification: str = Field(description="A brief justification for the scores, referencing specific steps the agent took.")

# Use a strong model for judging
judge_llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0).with_structured_output(ProcessLogicEvaluation)

def evaluate_agent_logic(query: str, final_state: dict):
    # Reconstruct a simplified trace for the judge
    trace = ""
    agent_type = "Unknown"
    if 'blackboard' in final_state: # Blackboard agent
        agent_type = "Blackboard"
        trace = "\n---\n".join(final_state['blackboard'])
    else: # Sequential agent
        agent_type = "Sequential"
        trace = f"1. News Report Generated: {final_state.get('news_report')}\n---\n2. Technical Report Generated: {final_state.get('technical_report')}\n---\n3. Financial Report Generated: {final_state.get('financial_report')}"

    prompt = f"""You are an expert judge of AI agent processes. Your task is to evaluate an agent's performance based on its generated content trace.

**User's Original Task:**
"{query}"

**Agent's Type:** {agent_type}
**Agent's Generated Content Trace:**
```
{trace}
```

**Evaluation Criteria:**
1.  **Instruction Following:** Did the agent respect the conditional logic in the user's task? (e.g., "either a technical analysis... or a financial analysis"). A high score means it followed the logic perfectly. A low score means it ignored it.
2.  **Process Efficiency:** Did the agent avoid doing unnecessary work? A high score means it only ran the required specialists. A low score means it ran specialists that the user's logic explicitly said to skip.

Based on the trace, provide your evaluation.
"""
    return judge_llm.invoke(prompt)

console.print("--- [bold]Evaluating Sequential Agent's Process[/bold] ---")
seq_agent_evaluation = evaluate_agent_logic(dynamic_query, final_seq_output)
console.print(seq_agent_evaluation.dict())

console.print("\n--- [bold]Evaluating Blackboard System's Process[/bold] ---")
bb_agent_evaluation = evaluate_agent_logic(dynamic_query, final_bb_output)
console.print(bb_agent_evaluation.dict())

--- Evaluating Sequential Agent's Process ---


{'instruction_following_score': 2, 'process_efficiency_score': 3, 'justification': "The agent failed to follow the core conditional instruction ('either/or'). Instead of choosing one path based on the news sentiment, it executed both the technical and financial analyses. This shows a lack of adherence to the user's logic and resulted in an inefficient process by performing unnecessary work."}



--- Evaluating Blackboard System's Process ---


{'instruction_following_score': 10, 'process_efficiency_score': 10, 'justification': 'The agent perfectly followed the user\'s conditional instructions. After the News Analyst reported positive sentiment, the system correctly chose to run the Technical Analyst and completely skipped the Financial Analyst. This demonstrates both flawless instruction following and optimal process efficiency, as no unnecessary work was performed.'}


**评估输出讨论：**
评判者的评分给出了清晰的定量结论：

- **顺序式代理** 将获得非常低的 `instruction_following_score`（指令遵循分数，例如 2/10），因为它公然忽略了“either/or”（非此即彼）的条件。其 `process_efficiency_score`（过程效率分数）也将很低（例如 3/10），因为它执行了明确不需要的完整分析。
- **黑板系统** 在两项分数上都将获得接近完美的分数（例如 10/10）。评判者将认可控制器的动态决策能力使系统能够精确遵循用户指令，并通过仅激活必要的专长代理来实现最高效率。

这项评估提供了确凿证据，证明对于复杂、涌现式问题——其中前进路径依赖于中间结果——黑板架构的灵活性远远优于刚性的、预定义的工作流程。

## 结论

在本笔记本中，我们实现了并修正了一个**黑板系统**，并展示了它相对于顺序式多代理架构的显著优势。通过引入共享内存（黑板）和一个智能的、有状态感知的**控制器**，我们创建了一个不仅具备协作能力，还具有适应性和机会主义特征的系统。

面对面对比显示，对于带有条件逻辑的任务，黑板系统在恰当的时机选择恰当的专长代理的能力，能够带来更高效且逻辑更严谨的过程。虽然它需要一个更复杂的控制器，但这种架构是为解决那些刚性线性工作流程无法有效应对的非结构化真实世界问题而设计的强大工具。